# Question solver

In [15]:
# imports
# imports

from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [22]:
# constants
MODEL_LLAMA = 'gpt-oss:20b'

In [23]:
from openai import OpenAI
MODEL = "gpt-oss:20b"
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

response = openai.chat.completions.create(
 model=MODEL,
 messages=[{"role": "user", "content": "What is 2 + 2?"}]
)

print(response.choices[0].message.content)

2 + 2 = 4.


In [24]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [25]:
# prompts

system_prompt = "You are a helpful technical tutor who answers questions about python code, software engineering, data science and LLMs"
user_prompt = "Please give a detailed explanation to the following question: " + question

In [26]:
# messages

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [27]:
# Get gpt-4o-mini to answer, with streaming

stream = openai.chat.completions.create(model=MODEL_LLAMA, messages=messages,stream=True)
    
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

### What the line actually does

python
yield from {book.get("author") for book in books if book.get("author")}


This line is usually written **inside a generator function** (a function that contains at least one `yield`).  
It consists of three logical parts:

| Part | Description | Example |
|------|-------------|---------|
| `{ ... }` | **Set comprehension** – constructs a `set` containing the values that come from the expressions inside the braces. | `{1, 2, 3}` |
| `book.get("author") for book in books if book.get("author")` | **Iterates over `books`** (which is expected to be an iterable of dictionaries). For each dictionary `book`, it looks up the value for the key `'author'`. The `if book.get("author")` clause filters out cases where that value is *falsy* (`None`, `''`, `0`, etc.). | If `books = [{'author': 'A'}, {'author': None}, {'author': 'B'}]` → the comprehension evaluates to `{'A', 'B'}`. |
| `yield from ...` | **Delegating yield** – tells the current generator to *yield* every value produced by the supplied iterable. In other words, it “drains” the iterable into the caller of the generator. | If the set is `{ 'A', 'B' }`, the generator will yield `'A'` and then `'B'` (order is arbitrary because a set has no defined order). |

So the whole line can be read as:

> *“Create a set of all non‑empty author names found in `books`, then yield each of those names one by one.”*

---

### Why use a **set** here?

1. **Uniqueness** – A set automatically removes duplicate values. If two books list the same author, the author will still appear only once in the yielded sequence.  
   python
   books = [{'author':'A'}, {'author':'A'}, {'author':'B'}]
   # set comprehension → {'A', 'B'}
   

2. **Performance** – Construction of a set and membership checks are `O(1)` on average, so for large lists of books the set comprehension is efficient.

---

### Why use `yield from` instead of a simple `for` loop?

`yield from` is a concise way to delegate the yielding of all items in an iterator to the calling code. It is semantically equivalent to:

python
for author in {book.get("author") for book in books if book.get("author") }:
    yield author


But `yield from` offers:

- **Cleaner syntax** – One line instead of a loop.
- **Proper delegation** – If the inner iterable returns a value with `return` (rare, but possible with custom iterators), that value would be returned by `yield from`. Sets don’t return a value, so it doesn’t matter here.

---

### Practical example

python
def unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

books = [
    {'title': 'Book 1', 'author': 'Alice'},
    {'title': 'Book 2'},                      # No author
    {'title': 'Book 3', 'author': 'Bob'},
    {'title': 'Book 4', 'author': 'Alice'},   # Duplicate
]

for author in unique_authors(books):
    print(author)


Possible output (order unspecified):


Alice
Bob


---

### Edge cases & pitfalls

| Issue | How it manifests | Mitigation |
|-------|------------------|------------|
| **Empty or missing author** | `book.get("author")` returns `None` or `''`, which are filtered out. | Nothing needed – the `if` clause does the job. |
| **Different truthy values** | If an author name is `'0'` or `'False'`, it would be considered falsy and removed. | Rare; if you truly need those values, replace the filter with `if book.get("author") is not None`. |
| **Large data sets** | Converting the whole list to a set first may use more memory than iterating lazily. | Use a generator expression inside `yield from`:

python
yield from (book["author"] for book in books if "author" in book)


but note that duplicates will be preserved unless you manage uniqueness yourself. |
| **Order matters (rare)** | A set has no guaranteed order; the yielded authors can appear in any order. | Use `sorted` or keep a `list` if the order matters. |

---

### Bottom line

The one‑liner

python
yield from {book.get("author") for book in books if book.get("author")}


is a very compact way of:

1. Building a **unique, non‑empty set** of authors from a collection of book dictionaries.
2. **Yielding** each author from that set to the caller of the generator.

It is both concise and readable for people familiar with set comprehensions and generator delegation. If you need the same logic but with guaranteed order or with a different data‑source, you can adapt the comprehension or replace `yield from` with an explicit loop.